## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Georgetown,MY,2021-04-05 15:39:47,5.4112,100.3354,84.20,78,40,4.61,light rain,0.0,0
1,1,Mauriti,BR,2021-04-05 15:44:23,-7.3892,-38.7744,94.19,30,73,7.43,broken clouds,0.0,0
2,2,Humaita,BR,2021-04-05 15:44:23,-7.5061,-63.0208,92.91,47,77,2.59,broken clouds,0.0,0
3,3,Bethel,US,2021-04-05 15:44:24,41.3712,-73.4140,55.40,33,1,16.11,clear sky,0.0,0
4,4,Barrow,US,2021-04-05 15:44:24,71.2906,-156.7887,-7.60,71,40,4.61,scattered clouds,0.0,0


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
while True:
    try:
        min_temp = float(input("Enter a minimum temperature preference. "))
        max_temp = float(input("Enter a maximum temperature preference. "))
        break
    except ValueError:
         print("Oops!  That was not a valid number.  Try again...")

Enter a minimum temperature preference. 23.78
Enter a maximum temperature preference. 100.6


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
pref_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
pref_cities_df

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Georgetown,MY,2021-04-05 15:39:47,5.4112,100.3354,84.20,78,40,4.61,light rain,0.0,0
1,1,Mauriti,BR,2021-04-05 15:44:23,-7.3892,-38.7744,94.19,30,73,7.43,broken clouds,0.0,0
2,2,Humaita,BR,2021-04-05 15:44:23,-7.5061,-63.0208,92.91,47,77,2.59,broken clouds,0.0,0
3,3,Bethel,US,2021-04-05 15:44:24,41.3712,-73.4140,55.40,33,1,16.11,clear sky,0.0,0
5,5,Parabel,RU,2021-04-05 15:44:25,58.6978,81.4825,35.06,89,100,11.77,overcast clouds,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,683,Kijang,KR,2021-04-05 15:48:08,35.2442,129.2139,48.20,57,0,5.75,clear sky,0.0,0
684,684,Port-Cartier,CA,2021-04-05 15:48:08,50.0334,-66.8654,41.00,100,90,10.36,mist,0.0,0
685,685,Guasdualito,VE,2021-04-05 15:48:09,7.2424,-70.7324,80.60,78,94,4.61,moderate rain,0.0,0
686,686,Barbastro,ES,2021-04-05 15:48:09,42.0356,0.1269,68.00,30,0,13.80,clear sky,0.0,0


In [5]:
# 4a. Determine if there are any empty rows.
pref_cities_df.isnull().sum()

Unnamed: 0                  0
City                        0
Country                     4
Date                        0
Lat                         0
Lng                         0
Max Temp                    0
Humidity                    0
Cloudiness                  0
Wind Speed                  0
Current Description         0
Rain inches (last 3 hrs)    0
Snow inches (last 3 hrs)    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = pref_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.count()

City                   620
Country                620
Max Temp               620
Current Description    620
Lat                    620
Lng                    620
Hotel Name             620
dtype: int64

In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    location = f"{lat}, {lng}"
# Adding the latitude and longitude to the params dictionary
    params["location"] = location
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print('Hotel found at , '+ hotel_df.loc[index, "City"])
    except (IndexError):
        print("Hotel not found. Skipping...")
print("all done")   

Hotel found at , Georgetown
Hotel found at , Mauriti
Hotel found at , Humaita
Hotel found at , Bethel
Hotel found at , Parabel
Hotel found at , Rikitea
Hotel found at , Punta Arenas
Hotel found at , Mataura
Hotel found at , Sao Filipe
Hotel found at , Sur
Hotel found at , Karaton
Hotel found at , Nanortalik
Hotel found at , Hobart
Hotel found at , Geraldton
Hotel found at , New Norfolk
Hotel found at , Bengkulu
Hotel found at , Bluff
Hotel found at , Jamestown
Hotel found at , Busselton
Hotel found at , Carnarvon
Hotel found at , Port Alfred
Hotel found at , Oranjestad
Hotel not found. Skipping...
Hotel found at , Folldal
Hotel found at , Lashio
Hotel found at , Vardo
Hotel found at , Hobyo
Hotel found at , Butaritari
Hotel found at , Nikolskoye
Hotel found at , Mahebourg
Hotel found at , East London
Hotel found at , Akyab
Hotel found at , Sovetskiy
Hotel found at , Pisco
Hotel not found. Skipping...
Hotel found at , Roald
Hotel found at , Ryotsu
Hotel found at , Pangody
Hotel found at

Hotel found at , Prainha
Hotel found at , Port Augusta
Hotel found at , Lesnoy
Hotel found at , Gull Lake
Hotel found at , Touros
Hotel found at , Gravelbourg
Hotel found at , Fort Nelson
Hotel found at , Yinchuan
Hotel found at , Ocean City
Hotel found at , Acari
Hotel found at , Nishihara
Hotel found at , Mao
Hotel not found. Skipping...
Hotel found at , Tambun
Hotel found at , Del Rio
Hotel found at , Channel-Port Aux Basques
Hotel found at , Lagoa
Hotel found at , Lake City
Hotel found at , Auki
Hotel found at , Worland
Hotel found at , Zhuhai
Hotel found at , Nouadhibou
Hotel found at , Sovetskaya Gavan
Hotel found at , Monduli
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel found at , Peniche
Hotel found at , Barsovo
Hotel found at , Atar
Hotel found at , Marsh Harbour
Hotel found at , Ulaanbaatar
Hotel found at , Roma
Hotel found at , Alofi
Hotel found at , Juneau
Hotel found at , Skjervoy
Hotel found at , Gwadar
Hotel not found. Skipping...
Hotel found at , Soyo

Hotel found at , Kijang
Hotel found at , Port-Cartier
Hotel found at , Guasdualito
Hotel found at , Barbastro
Hotel found at , Paamiut
all done


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.loc[(hotel_df['Hotel Name'] != "")]
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Georgetown,MY,84.20,light rain,5.4112,100.3354,Cititel Penang
1,Mauriti,BR,94.19,broken clouds,-7.3892,-38.7744,Hotel Pedra da Letra
2,Humaita,BR,92.91,broken clouds,-7.5061,-63.0208,Corrêa Hotel
3,Bethel,US,55.40,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
5,Parabel,RU,35.06,overcast clouds,58.6978,81.4825,Housing Complex Gazprom Transgaz Tomsk
...,...,...,...,...,...,...,...
683,Kijang,KR,48.20,clear sky,35.2442,129.2139,Samyang Beach Villa
684,Port-Cartier,CA,41.00,mist,50.0334,-66.8654,Le Q'Artier
685,Guasdualito,VE,80.60,moderate rain,7.2424,-70.7324,Hotel Cuibas
686,Barbastro,ES,68.00,clear sky,42.0356,0.1269,Gran Hotel Ciudad De Barbastro


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [3]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [1]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

NameError: name 'hotel_df' is not defined